In [ ]:
#pip install nba_api

In [17]:
import pandas as pd
from nba_api.stats.static import players
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.endpoints import playerdashboardbygamesplits

In [2]:
player_dict = players.find_players_by_full_name("LeBron James")
print(player_dict)

[{'id': 2544, 'full_name': 'LeBron James', 'first_name': 'LeBron', 'last_name': 'James', 'is_active': True}]


In [3]:
player_id = 2544  # LeBron James
career = playercareerstats.PlayerCareerStats(player_id=player_id)
career_df = career.get_data_frames()[0]
career_df.head()

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,2544,2003-04,00,1610612739,CLE,19.0,79,79,3120.0,622,...,0.754,99,333,432,465,130,58,273,149,1654
1,2544,2004-05,00,1610612739,CLE,20.0,80,80,3388.0,795,...,0.750,111,477,588,577,177,52,262,146,2175
2,2544,2005-06,00,1610612739,CLE,21.0,79,79,3361.0,875,...,0.738,75,481,556,521,123,66,260,181,2478
3,2544,2006-07,00,1610612739,CLE,22.0,78,78,3190.0,772,...,0.698,83,443,526,470,125,55,250,171,2132
4,2544,2007-08,00,1610612739,CLE,23.0,75,74,3027.0,794,...,0.712,133,459,592,539,138,81,255,165,2250


In [4]:
from nba_api.stats.endpoints import leaguedashplayerstats

stats = leaguedashplayerstats.LeagueDashPlayerStats(season='2023-24')
stats_df = stats.get_data_frames()[0]
stats_df.head()

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK
0,1630639,A.J. Lawson,A.J.,1610612742,DAL,23.0,42,27,15,0.643,...,428,211,148,415,385,192,400,257,38,405
1,1631260,AJ Green,AJ,1610612749,MIL,24.0,56,35,21,0.625,...,410,121,223,375,313,145,369,257,38,343
2,1631100,AJ Griffin,AJ,1610612737,ATL,20.0,20,8,12,0.400,...,451,121,77,519,450,396,480,257,38,469
3,203932,Aaron Gordon,Aaron,1610612743,DEN,28.0,73,49,24,0.671,...,79,520,457,52,93,13,79,54,38,87
4,1628988,Aaron Holiday,Aaron,1610612745,HOU,27.0,78,39,39,0.500,...,370,335,419,238,216,123,243,257,38,233


In [5]:
stats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572 entries, 0 to 571
Data columns (total 66 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PLAYER_ID              572 non-null    int64  
 1   PLAYER_NAME            572 non-null    object 
 2   NICKNAME               572 non-null    object 
 3   TEAM_ID                572 non-null    int64  
 4   TEAM_ABBREVIATION      572 non-null    object 
 5   AGE                    572 non-null    float64
 6   GP                     572 non-null    int64  
 7   W                      572 non-null    int64  
 8   L                      572 non-null    int64  
 9   W_PCT                  572 non-null    float64
 10  MIN                    572 non-null    float64
 11  FGM                    572 non-null    int64  
 12  FGA                    572 non-null    int64  
 13  FG_PCT                 572 non-null    float64
 14  FG3M                   572 non-null    int64  
 15  FG3A  

In [9]:
# Get player ID for Stephen Curry
player = players.find_players_by_full_name("Stephen Curry")[0]
player_id = player['id']

# Get 2023-24 regular season game logs
gamelog = playergamelog.PlayerGameLog(player_id=player_id, season='2023-24', season_type_all_star='Regular Season')
log_df = gamelog.get_data_frames()[0]

# Convert GAME_DATE to datetime and sort
log_df['GAME_DATE'] = pd.to_datetime(log_df['GAME_DATE'])
log_df = log_df.sort_values('GAME_DATE')

C:\Users\ericz\AppData\Local\Temp\ipykernel_15816\2773911200.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  log_df['GAME_DATE'] = pd.to_datetime(log_df['GAME_DATE'])


In [10]:
# Tag games as Home or Away
log_df['HOME'] = log_df['MATCHUP'].apply(lambda x: 'Away' if '@' in x else 'Home')

# Group and average by HOME/Away
location_stats = log_df.groupby('HOME')[['PTS', 'REB', 'AST', 'FG_PCT', 'FG3_PCT']].mean().round(2)
print(location_stats)

        PTS   REB   AST  FG_PCT  FG3_PCT
HOME                                    
Away  25.66  4.61  5.58    0.44     0.39
Home  27.25  4.31  4.64    0.45     0.42


In [11]:
# Extract opponent team from MATCHUP column
log_df['OPPONENT'] = log_df['MATCHUP'].apply(lambda x: x.split()[-1])

# Group by opponent and calculate mean stats
opponent_stats = log_df.groupby('OPPONENT')[['PTS', 'REB', 'AST', 'FG_PCT']].mean().round(2)
print(opponent_stats)

            PTS   REB    AST  FG_PCT
OPPONENT                            
ATL       42.50  4.50   6.00    0.51
BKN       33.00  4.00   2.00    0.57
BOS       18.50  2.00   4.50    0.34
CHA       19.00  1.50   4.50    0.43
CHI       21.00  4.00   5.00    0.31
CLE       29.00  2.50   3.00    0.44
DAL       22.00  5.00   6.33    0.34
DEN       22.75  3.00   4.50    0.37
DET       30.00  4.00   4.00    0.42
HOU       28.33  6.00   4.33    0.53
IND       33.50  6.50   3.50    0.53
LAC       26.50  4.75   7.00    0.42
LAL       33.00  4.25   7.00    0.57
MEM       20.00  4.00   5.67    0.49
MIA       15.00  3.00   5.50    0.33
MIL       29.00  8.00   5.00    0.56
MIN       34.50  4.50   3.00    0.43
NOP       30.00  3.33   5.33    0.50
NYK       29.00  8.00   3.00    0.41
OKC       29.67  7.00   4.33    0.49
ORL       26.50  3.00   8.00    0.47
PHI       23.00  5.50   4.00    0.50
PHX       24.25  5.50   4.25    0.40
POR       21.75  4.25   4.75    0.40
SAC       31.00  6.25   3.75    0.53
S

In [12]:
# Standard deviation of core stats (lower = more consistent)
consistency = log_df[['PTS', 'REB', 'AST', 'FG_PCT']].std().round(2)
print("Stat consistency (lower is more consistent):")
print(consistency)

Stat consistency (lower is more consistent):
PTS       9.58
REB       2.38
AST       2.42
FG_PCT    0.13
dtype: float64


In [15]:
season_stats = leaguedashplayerstats.LeagueDashPlayerStats(season='2023-24')
df = season_stats.get_data_frames()[0]

# Select only the columns you want
selected_columns = [
    'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ABBREVIATION',
    'GP', 'MIN', 'PTS', 'REB', 'AST', 'FG_PCT', 'FG3_PCT', 'FT_PCT', 'TOV', 'PLUS_MINUS'
]

clean_df = df[selected_columns]
clean_df.tail()

,PLAYER_ID,PLAYER_NAME,TEAM_ABBREVIATION,GP,MIN,PTS,REB,AST,FG_PCT,FG3_PCT,FT_PCT,TOV,PLUS_MINUS
567,203897,Zach LaVine,CHI,25,871.861667,487,129,98,0.452,0.349,0.854,52,-92
568,1630285,Zavier Simpson,MEM,7,161.383333,42,20,25,0.315,0.294,0.750,10,-29
569,1630192,Zeke Nnaji,DEN,58,576.120000,186,125,32,0.463,0.261,0.677,27,-168
570,1630533,Ziaire Williams,MEM,51,1037.681667,420,180,75,0.397,0.307,0.827,66,-247
571,1629627,Zion Williamson,NOP,70,2206.923333,1601,406,352,0.570,0.333,0.702,193,143


In [18]:
# Find player ID (e.g. Stephen Curry)
player = players.find_players_by_full_name("Stephen Curry")[0]
player_id = player['id']

# Get home/away split stats
split = playerdashboardbygamesplits.PlayerDashboardByGameSplits(player_id=player_id, season='2023-24')
home_away_df = split.get_data_frames()[1]  # index 1 = HOME/AWAY table

print(home_away_df[['GROUP_VALUE', 'GP', 'PTS', 'REB', 'AST', 'FG_PCT']])

   GROUP_VALUE  GP   PTS  REB  AST  FG_PCT
0   First Half  74   879  174  209   0.449
1  Second Half  73  1034  154  169   0.446
2     Overtime   5    43    2    1   0.583
